In [1]:
# Load required libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import fiona
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from shapely.ops import unary_union
from shapely.errors import TopologicalError
from unidecode import unidecode
import glob
import csv
from datetime import datetime
import dask.dataframe as dd
import dask_geopandas as dg
from dask.distributed import Client
import gc
import re

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.2-CAPI-1.17.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_60272\2701944437.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https:

In [6]:
# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
outdata_f = os.path.join(indata_f, 'OutputData')
if not os.path.exists(outdata_f):
    # Create the folder if it doesn't exist
    os.makedirs(outdata_f)

# 0 PREPARE A LOG FILE FOR QC
log_file = 'log_GQA_Step1.csv'
log_path = os.path.join(outdata_f, log_file)

# Initialize Dask client
client = Client()

# Define engines
engines = {
    'fiona': {'engine': 'fiona'},
    'pyogrio': {'engine': 'pyogrio'},
    'pyogrio+arrow': {'engine': 'pyogrio', 'use_arrow': True}
          
}


c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 62870 instead
  warnings.warn(


In [7]:
# 1 READ URBAN CENTRES
# Read shapefile
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp')
# Read the GeoPackage file
uc = gpd.read_file(uc_file_path)
uc['CNTR_CODE'].fillna('AA', inplace=True)

# Select cities for processing in this batch
###uc_sel = uc.query('Batch==1.0 & CNTR_CODE != "SE"')
uc_sel = uc.query('Batch>0.0')
uc_sel = uc_sel.sort_values(by='CNTR_CODE')

# Read table to list the cities to process using urban centre code
cities_ls = uc_sel.HDENS_CLST.tolist()
len(cities_ls)

240

In [8]:
# Final GQAs
QGA_Final_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final'
# Read table with HDENS Urban centres information and Agglomerations link
HDENS_AGGL_tbl = pd.read_csv(r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\UrbanCentres_Agglomerations_csv.csv')
# Join uc code field to this table
HDENS_AGGL_tbl = HDENS_AGGL_tbl.merge(uc[['POPL_2021', 'HDENS_CLST']], on='POPL_2021')

# 1 UA DATA FOLDER
ua_data_f = r'A:\Copernicus\UrbanAtlas\UrbanAtlas\UA2018'

# 2 READ NOISE DATA
# Load agglomerations delineations
agls_file_path = os.path.join(indata_f, 'NoiseData', 'DF1_5_Agglomerations_20240429.gpkg')

# Read the GeoPackage file
agls = gpd.read_file(agls_file_path, layer = 'dbo.DF15_AgglomerationSource_Valid_LatestDelivery', 
                     **engines['pyogrio+arrow'],columns=['agglomerationId_identifier', 'agglomerationName_nameEng', 'geometry'])

# 3 TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

In [14]:
# Output folder
GQA_UA_f = os.path.join(QGA_Final_path, 'GQA_UA')
if not os.path.exists(GQA_UA_f):
    # Create the folder if it doesn't exist
    os.makedirs(GQA_UA_f)

In [11]:
cities_ls.sort()
cities_ls[-1]

'GEOSTAT21_945'

In [20]:
for uc_code in cities_ls:
    print(uc_code)
    inGQA = os.path.join(QGA_Final_path, f'{uc_code}_finalGQA.shp')
    if os.path.exists(inGQA):
        GQA = gpd.read_file(inGQA)
        print("GQA loaded")
        outGQA_path = os.path.join(GQA_UA_f, f'{uc_code}_GQA_UA.shp')
        if not os.path.exists(outGQA_path):
            if 'code_2018' in GQA.columns:
                GQA.to_file(outGQA_path, driver='ESRI Shapefile')
                print("QGA saved to outfolder") 
            else:                
                ua_urbc_agl_green_path = os.path.join(outdata_f, 'Green_inUCUA_Aglos', f'{uc_code}_greenUCUANCM.shp')
                if not os.path.exists(ua_urbc_agl_green_path):
                    print("ua_urbc_agl_green not found")    
                else:
                    ua_urbc_agl_green = gpd.read_file(ua_urbc_agl_green_path)
                    print("ua_urbc_agl_green loaded")  
                    outGQA = gpd.overlay(GQA, ua_urbc_agl_green, how='intersection')
                    print("QGA and ua intersected") 
                    outGQA.to_file(outGQA_path, driver='ESRI Shapefile')
                    print("QGA saved to outfolder") 
    else:
        print(f"GQA not found - {inGQA}")

GEOSTAT21_002
GQA loaded
GEOSTAT21_003
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_004
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_005
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_006
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_007
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_011
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_012
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_013
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_016
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_018
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_021
G

Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_60272\2285465470.py:19: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  outGQA = gpd.overlay(GQA, ua_urbc_agl_green, how='intersection')


QGA saved to outfolder
GEOSTAT21_154
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_161
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_168
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_171
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_173
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_177
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_178
GQA loaded
ua_urbc_agl_green loaded


Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_60272\2285465470.py:19: UserWarning: `keep_geom_type=True` in overlay resulted in 1 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  outGQA = gpd.overlay(GQA, ua_urbc_agl_green, how='intersection')


QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_181
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_184
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_186
GQA loaded
ua_urbc_agl_green loaded


...

QGA saved to outfolder
GEOSTAT21_549
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_550
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_557
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_558
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_561
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_569
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_580
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_588
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_595
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_597
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_620
GQA

c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\pygeos\set_operations.py:129: RuntimeWarning: divide by zero encountered in intersection
  return lib.intersection(a, b, **kwargs)
c:\Users\ubach\.conda\envs\geospatial_ETC_DI_v4\lib\site-packages\pygeos\set_operations.py:129: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_903
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_904
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_905
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_906
GQA loaded
ua_urbc_agl_green loaded


Q:\UserTemp\ubach\AppData\Local\Temp\3\ipykernel_60272\2285465470.py:19: UserWarning: `keep_geom_type=True` in overlay resulted in 4 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  outGQA = gpd.overlay(GQA, ua_urbc_agl_green, how='intersection')


QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_907
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_911
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_912
GQA not found - P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final\GEOSTAT21_912_finalGQA.shp
GEOSTAT21_914
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_919
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_922
GQA loaded
ua_urbc_agl_green loaded
QGA and ua intersected
QGA saved to outfolder
GEOSTAT21_925
GQA loaded
QGA saved to outfolder
GEOSTAT21_941
GQA not found - P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\OutputData\step2_GQA_Final\GEOSTAT21_941_finalGQA.shp
GEOSTAT21_943
GQA not found - P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract